In [56]:
#Importing the necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [57]:
df = pd.read_excel("df_OHE.xlsx") # reading the file and printing the first 5 rows 
df.head()

,fss__class__score,wex_tenure,money_code_to_ttl_spend,commercial__credit__score,ai_rg,fuel_amt_by_ttl_spend_12m_overall,years__in__business,cash_amt_by_ttl_spend_12m_overall,financial__stress__score,misc_amt_by_ttl_spend_12m_overall,paydex,has_security_deposit,bankruptcy__found,risk_grade__c,Event_Rate
0,3,29,0.082600,567,4,0.915819,2,0.001067,1495,0.000514,80,0,0,8,0
1,3,269,0.016501,547,2,0.981481,39,0.001885,1487,0.000133,80,1,0,8,0
2,4,54,0.000000,492,2,0.987366,5,0.000000,1408,0.012634,75,0,0,8,0
3,2,93,0.000000,577,2,1.000000,39,0.000000,1521,0.000000,80,0,0,8,0
4,4,101,0.000000,316,2,1.000000,13,0.000000,1431,0.000000,67,0,0,8,0


In [58]:
df.info() # getting basic information about the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82852 entries, 0 to 82851
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   fss__class__score                  82852 non-null  int64  
 1   wex_tenure                         82852 non-null  int64  
 2   money_code_to_ttl_spend            82852 non-null  float64
 3   commercial__credit__score          82852 non-null  int64  
 4   ai_rg                              82852 non-null  int64  
 5   fuel_amt_by_ttl_spend_12m_overall  82852 non-null  float64
 6   years__in__business                82852 non-null  int64  
 7   cash_amt_by_ttl_spend_12m_overall  82852 non-null  float64
 8   financial__stress__score           82852 non-null  int64  
 9   misc_amt_by_ttl_spend_12m_overall  82852 non-null  float64
 10  paydex                             82852 non-null  int64  
 11  has_security_deposit               82852 non-null  int

In [59]:
df.describe() # describing the data

,fss__class__score,wex_tenure,money_code_to_ttl_spend,commercial__credit__score,ai_rg,fuel_amt_by_ttl_spend_12m_overall,years__in__business,cash_amt_by_ttl_spend_12m_overall,financial__stress__score,misc_amt_by_ttl_spend_12m_overall,paydex,has_security_deposit,bankruptcy__found,risk_grade__c,Event_Rate
count,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000,82852.000000
mean,4.375404,39.407075,0.049412,486.059105,3.263603,0.905506,501.509776,0.016711,1431.561568,0.028371,60.565020,0.030911,0.001424,5.553891,0.024634
std,1.872481,47.333320,0.185122,94.833703,1.262625,0.218718,858.044091,0.087732,210.640311,0.091024,15.839444,0.173076,0.037712,5.519404,0.155009
min,1.000000,-47.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-0.201353,0.000000,-0.129234,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,8.000000,0.000000,450.000000,2.000000,0.943461,3.000000,0.000000,1449.000000,0.000000,50.000000,0.000000,0.000000,3.000000,0.000000
50%,4.000000,21.000000,0.000000,476.000000,3.000000,0.973068,9.000000,0.000000,1450.000000,0.013311,50.000000,0.000000,0.000000,8.000000,0.000000
75%,7.000000,52.000000,0.000000,557.000000,4.000000,0.998094,97.000000,0.000000,1491.000000,0.029844,80.000000,0.000000,0.000000,8.000000,0.000000
max,7.000000,364.000000,1.201353,670.000000,8.000000,1.129234,2000.000000,1.000000,1706.000000,1.002268,89.000000,1.000000,1.000000,995.000000,1.000000


#### Attempting Multiple Models

In [60]:
X = df.iloc[:,:-1]  #independent variables
y = df.iloc[:,-1]   # dependent variable

In [61]:
#creating objects of all required models and storing them in the list

models=[]
models.append(("LogisticRegression",LogisticRegression(solver='sag',max_iter=2000)))
models.append(("DecisionTree",DecisionTreeClassifier()))
models.append(("RandomForest",RandomForestClassifier()))
models.append(("KNeighbors",KNeighborsClassifier()))
models.append(('GaussianNB',GaussianNB()))
models.append(("SVC",SVC()))
models.append(('AdaBoost',AdaBoostClassifier()))
models.append(('GradientBoosting',GradientBoostingClassifier()))

models

[('LogisticRegression', LogisticRegression(max_iter=2000, solver='sag')),
 ('DecisionTree', DecisionTreeClassifier()),
 ('RandomForest', RandomForestClassifier()),
 ('KNeighbors', KNeighborsClassifier()),
 ('GaussianNB', GaussianNB()),
 ('SVC', SVC()),
 ('AdaBoost', AdaBoostClassifier()),
 ('GradientBoosting', GradientBoostingClassifier())]

In [62]:
# creating 2 lists one for name and other for model object. for ex:name = "LogisticRegression" and model= LogisticRegression()

results = []
names = []
for name,model in models:
    result = cross_val_score(model, X, y,  cv=10)  # computing cross validation score for every model
    names.append(name)
    results.append(result)

#     for i in range(len(names)):
#         print(names[i],':',round(results[i].mean(),2),':',round(results[i].std(),4)) # getting mean and std for every model

In [63]:
#creating final dataframe on the basis of three lists modelname , mean and standard deviation

model_name=[]
model_cvmean=[]
model_cvstd=[]
for i in range(len(names)):
    model_name.append(names[i])
    model_cvmean.append(round(results[i].mean(),2))
    model_cvstd.append(round(results[i].std(),4))

final_dataframe=pd.DataFrame.from_dict({'Name' : model_name, 'Mean_of_K-Folds_CVScore' : model_cvmean, 'Std_CVKFolds' : model_cvstd})
final_dataframe.to_excel('Stratified_KFOLD.xlsx',index=False)
#print(names[i],':',round(results[i].mean(),2),':',round(results[i].std(),4)) # getting mean and std for every model

### Creating Pipeline

In [64]:
# from sklearn.pipeline import Pipeline

# model_pipeline = Pipeline(steps=[('Model Objects', defining_model_objects()), 
#                                  ('Cross Validation Score',calculating_CVScore()),
#                                  ('final_result', creating_final_df())
#                                  ])
# # fit the pipeline with the training data
# model_pipeline.fit(X,y)

# # # predict target values on the training data
# # model_pipeline.predict(train_x)